In [1]:
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import json
import os
from scipy.optimize import curve_fit
from collections import defaultdict
import ast
import re
from itertools import combinations
import itertools
from matplotlib import font_manager
import matplotlib.dates as mdates


# 指定字体
font_path = "C:/Windows/Fonts/simsun.ttc"  # SimSun字体路径
font_prop = font_manager.FontProperties(fname=font_path)

plt.rcParams["font.sans-serif"] = [font_prop.get_name()]
plt.rcParams["axes.unicode_minus"] = False  # 解决负号显示问题
plt.rcParams["font.size"] = 10.5  # 设置五号字体大小


In [2]:
# 读取三个xlsx文件，假设文件名分别为file1.xlsx, file2.xlsx, file3.xlsx
file_paths = ["D:/研究生/毕业论文/数据/主播数据/jieguo_1000(2024.03.16-2024.04.14)/info.xlsx", 
             "D:/研究生/毕业论文/数据/主播数据/jieguo_500-1000(2024.03.16-2024.04.14)/info.xlsx",
             "D:/研究生/毕业论文/数据/主播数据/jieguo_100-500(2024.03.16-2024.04.14)/info.xlsx",
            "D:/研究生/毕业论文/数据/主播数据/jieguo_10-100(2024.03.16-2024.04.14)/info.xlsx"
             ]


# 删除非带货主播

In [ ]:
file_paths = ["D:/研究生/毕业论文/数据/主播数据/livelinks_1000(2024.03.16-2024.04.14).csv", 
             "D:/研究生/毕业论文/数据/主播数据/livelinks_500-1000(2024.03.16-2024.04.14).csv",
             "D:/研究生/毕业论文/数据/主播数据/livelinks_100-500(2024.03.16-2024.04.14).csv",
            "D:/研究生/毕业论文/数据/主播数据/livelinks_10-100(2024.03.16-2024.04.14).csv"
             ]

In [ ]:

# 读取xlsx文件
df = pd.read_csv('D:/研究生/毕业论文/数据/主播数据/livelinks_10-100(2024.03.16-2024.04.14).csv', encoding='gbk')
# 筛选is_take_product为FALSE的行。gbk读取为布尔值，而不是字符串。
filtered_df = df[df['is_take_product'] == False]
# 获取要删除的room_id列表
room_ids_to_delete = filtered_df['room_id']
room_ids_to_delete = room_ids_to_delete.str.replace('\t', '')
data_folder = 'D:/研究生/毕业论文/数据/主播数据/meichangzhibodata_10-100(2024.03.16-2024.04.14)'
# 遍历文件夹中的文件
for filename in os.listdir(data_folder):
    # 提取文件名的前缀部分（去掉文件扩展名）
    file_prefix = os.path.splitext(filename)[0]  # 获取不含扩展名的文件名前缀

    # 检查提取的文件前缀是否在room_ids_to_delete中
    if file_prefix in room_ids_to_delete.values:
        # 构建完整的文件路径
        file_path = os.path.join(data_folder, filename)
        try:
            # 删除文件
            os.remove(file_path)
            print(f"Deleted file: {file_path}")
        except Exception as e:
            print(f"Error deleting file {file_path}: {e}")
    else:
        print(f"Skipped file: {filename} (room_id not in list)")

1135个主播，32413条直播

# 一个月内的频数分布统计

In [4]:
# 创建一个空DataFrame用于存储所有数据
df_all = pd.DataFrame()

# 循环读取每个文件并合并数据
# 假设文件名中包含等级信息，从文件路径中提取等级
# for file_path in file_paths:
#     if '10-100' in file_path:
#         level = 'Level 4'
#     elif '100-500' in file_path:
#         level = 'Level 3'
#     elif '500-1000' in file_path:
#         level = 'Level 2'
#     elif '1000' in file_path:
#         level = 'Level 1'
#     else:
#         level = 'Unknown'  # 处理未知等级的文件
# 循环读取每个文件并合并数据
# 假设文件名中包含等级信息，从文件路径中提取等级
for file_path in file_paths:
    if '10-100' in file_path:
        level = '3'
    elif '100-500' in file_path:
        level = '2'
    elif '500-1000' in file_path:
        level = '1'
    elif '1000' in file_path:
        level = '1'
    else:
        level = 'Unknown'  # 处理未知等级的文件
    # 读取xlsx文件
    df = pd.read_excel(file_path)
    
    # 添加等级列
    df['level'] = level
    
    # 合并数据
    df_all = pd.concat([df_all, df], ignore_index=True)

# 将 begin_time 和 room_finish_time 列的时间戳转换为 pandas 的日期时间格式
df_all['begin_time'] = pd.to_datetime(df_all['begin_time'], unit='s')
df_all['room_finish_time'] = pd.to_datetime(df_all['room_finish_time'], unit='s')

# 将日期时间列的时区从 UTC 转换为北京时间
df_all['begin_time'] = df_all['begin_time'].dt.tz_localize('UTC').dt.tz_convert('Asia/Shanghai')
df_all['room_finish_time'] = df_all['room_finish_time'].dt.tz_localize('UTC').dt.tz_convert('Asia/Shanghai')

# 提取日期作为新的列
df_all['date'] = df_all['begin_time'].dt.date

# 按照日期和等级等特征进行分组统计
result = df_all.groupby(['date', 'level']).size().unstack(fill_value=0)

In [11]:
# 绘制图表
plt.figure(figsize=(8, 6), dpi=400)  # 设置图表尺寸和分辨率
sns.lineplot(data=result, markers=True, dashes=False)
# plt.title('每日直播间频率分布（按主播等级划分）', fontproperties=font_prop)
plt.xlabel('日期', fontweight='bold', fontproperties=font_prop)
plt.ylabel('频率', fontweight='bold', fontproperties=font_prop)
plt.xticks(rotation=45)
legend = plt.legend(title='主播等级', fontsize=10.5, loc='best', prop={'family': 'SimSun'})

# 使用 set_font_properties 方法来设置图例标题的字体
legend.get_title().set_font_properties(font_prop)
plt.tight_layout()

# 保存图为PDF文件
plt.savefig('D:/研究生/毕业论文/草稿/图片/date_level.png')
# 显示图表
plt.show()


In [14]:
# 创建画布和子图
fig, axes = plt.subplots(3, 1, figsize=(8, 6), dpi=200)

# 将二维数组展平以便迭代
axes = axes.flatten()

# 循环绘制每列数据的子图
for i, column in enumerate(result.columns):
    sns.lineplot(data=result[column], ax=axes[i], marker='o', dashes=False)
    axes[i].set_title(f'{column} Frequency by Date')
    axes[i].set_xlabel('Date')
    axes[i].set_ylabel('Frequency')
    axes[i].tick_params(axis='x', rotation=45)
    axes[i].legend([column], loc='upper right')

# 调整布局
plt.tight_layout()

# 显示图表
plt.show()


在一个月内，头部主播每天的直播很稳定，非头部主播每天的直播很不稳定。但是单独看每类主播都不稳定，可能是因为非头部的基数太大造成变化幅度很大，因此放在一张图的时候，会显得头部主播的每天直播就很稳定。

# 一周内的频数分布统计

In [7]:
# 假设 df_all 是已经处理好的数据框，包含了 begin_time、room_finish_time、level 等列
# 请确保 df_all 包含这些列，并已将日期时间转换为 pandas 的日期时间格式，并添加了 level 列

# 创建日期对应的星期几映射
day_of_week_map = {
    0: '星期一',
    1: '星期二',
    2: '星期三',
    3: '星期四',
    4: '星期五',
    5: '星期六',
    6: '星期日'
}

# 获取日期对应的星期几函数
def get_day_of_week(date):
    return day_of_week_map[date.dayofweek]

# 添加星期几列
df_all['day_of_week'] = df_all['begin_time'].apply(get_day_of_week)

# 按照星期几和等级等特征进行分组统计
result = df_all.groupby(['day_of_week', 'level']).size().unstack(fill_value=0)

# 定义按照顺序排序的星期几列表
week_order = ['星期一', '星期二', '星期三', '星期四', '星期五', '星期六', '星期日']

# 按照指定顺序重新排列索引
result = result.reindex(index=week_order)

In [10]:
# 设置数据
weekdays = ['星期一', '星期二', '星期三', '星期四', '星期五', '星期六', '星期日']
levels = ['1', '2', '3']
data = result.loc[:, levels]

# 自定义颜色列表
colors = [(144/255, 201/255, 231/255), (33/255, 158/255, 188/255), (19/255, 103/255, 131/255)]

# 创建画布和子图
fig, axs = plt.subplots(3, 1, figsize=(8, 6), dpi=400)

# 遍历每个子图位置以及对应的水平索引和垂直索引
for i, (level, ax) in enumerate(zip(levels, axs.flat)):
    # 绘制柱形图
    ax.bar(np.arange(len(weekdays)), data[level], color=colors[i])
    
    # 设置标题和坐标轴标签
    ax.set_title(f'主播等级 {i+1}', fontweight='bold', fontproperties=font_prop)
    ax.set_ylabel('频率', fontweight='bold', fontproperties=font_prop)
    
    # 只在最后一个子图上设置 x 轴标签
    if i == len(levels) - 1:
        ax.set_xticks(np.arange(len(weekdays)))
        ax.set_xticklabels(weekdays, fontproperties=font_prop)
        ax.set_xlabel('星期', fontproperties=font_prop)
    else:
        ax.set_xticks([])  # 移除其它子图的 x 轴刻度

# 调整子图之间的间距
plt.tight_layout()

# 保存图为PDF文件
plt.savefig('D:/研究生/毕业论文/草稿/图片/weekday_level.png')

# 显示图形
plt.show()


可以看出，周末和非周末之间是非常不一样的直播频数，不仅是头部，还是非头部。因此需要分开讨论两种情况。

# 一天内的频数分布统计

In [12]:
# 定义时间段划分函数
def get_time_period(hour):
    if 6 <= hour < 12:
        return '上午'
    elif 12 <= hour < 18:
        return '下午'
    elif 18 <= hour < 24:
        return '晚上'
    else:
        return '凌晨'

# 创建一个空的 DataFrame 用于存储时间段统计结果
time_period_counts = pd.DataFrame()

# 遍历每个文件中的数据
for index, row in df_all.iterrows():
    # 获取开始时间的小时并根据小时获取时间段
    begin_hour = row['begin_time'].hour
    time_period = get_time_period(begin_hour)
    
    # 将时间段添加到原始数据中的新列 time_period 中
    df_all.loc[index, 'time_period'] = time_period

# 按照时间段和等级分组统计
result_time_periods = df_all.groupby(['time_period', 'level']).size().unstack(fill_value=0)


In [13]:
# 定义按照顺序排序的星期几列表
day_order = ['凌晨', '上午', '下午', '晚上']

# 按照指定顺序重新排列索引
result_time_periods = result_time_periods.reindex(index=day_order)

In [14]:
# 定义颜色
colors = [(144/255, 201/255, 231/255), (33/255, 158/255, 188/255), (19/255, 103/255, 131/255)]

# 定义时间段顺序（确保和 result_time_periods.index 一致）
day_order = ['凌晨', '上午', '下午', '晚上']  # 你需要根据实际时间段顺序修改此列表

# 创建 3x1 的子图网格
fig, axs = plt.subplots(3, 1, figsize=(8, 6), dpi=400)

# 在每个子图中绘制数据
for i, ax in enumerate(axs.flat):
    # 获取对应的时间段数据
    time_period_data = result_time_periods.iloc[:, i]
    
    # 绘制柱形图
    ax.bar(time_period_data.index, time_period_data, color=colors[i])
    
    # 设置子图标题
    ax.set_title(f'主播等级 {i+1}', fontweight='bold', fontproperties=font_prop)
    
    # 设置子图坐标轴标签
    ax.set_ylabel('频率', fontweight='bold', fontproperties=font_prop)
    
    # 设置 x 轴刻度和标签
    if i == len(axs.flat) - 1:  # 仅在最后一个子图设置 x 轴刻度和标签
        ax.set_xticks(np.arange(len(day_order)))
        ax.set_xticklabels(day_order, fontproperties=font_prop)
        ax.set_xlabel('时间段', fontproperties=font_prop)
    else:
        ax.set_xticks([])  # 移除其他子图的 x 轴刻度和标签

# 调整子图布局
plt.tight_layout()

# 保存图为PDF文件
plt.savefig('D:/研究生/毕业论文/草稿/图片/period_level.png')

# 显示图形
plt.show()


可以看出，大部分的直播集中在非睡眠时间，同时直播间的热度呈现逐渐上升的趋势。但是不同等级的主播重点直播时间段有所不同。

# 每小时内的频数分布

In [15]:
# 添加 hour 列，根据 begin_time 提取小时
df_all['hour'] = df_all['begin_time'].dt.hour

# 按照小时和等级分组统计
result_hours = df_all.groupby(['hour', 'level']).size().unstack(fill_value=0)


In [16]:
# 设置画布和子图数量
num_levels = result_hours.shape[1]
fig, axs = plt.subplots(num_levels, 1, figsize=(8, 6), dpi=400)

# 如果只有一个等级时，axs 不是列表，需要转换为列表
if num_levels == 1:
    axs = [axs]

# 遍历每个等级并绘制在对应的子图上
for i, level in enumerate(result_hours.columns):
    axs[i].plot(result_hours.index, result_hours[level], marker='o')
    axs[i].set_title(f'主播等级 {level}', fontproperties=font_prop)  # 使用中文字体显示标题
    axs[i].set_ylabel('频数', fontproperties=font_prop)
    # axs[i].grid(True)

# 设置公共的 x 轴标签
axs[-1].set_xlabel('小时', fontproperties=font_prop)

# 调整布局
plt.tight_layout()

# 保存图为PDF文件
plt.savefig('D:/研究生/毕业论文/草稿/图片/hour_level.png')

plt.show()



# 每小时直播数据的频数分布统计

In [5]:
# 读取三个xlsx文件，假设文件名分别为file1.xlsx, file2.xlsx, file3.xlsx
file_multi_paths = ["D:/研究生/毕业论文/数据/主播数据/jieguo_1000(2024.03.16-2024.04.14)/multi.xlsx", 
             "D:/研究生/毕业论文/数据/主播数据/jieguo_500-1000(2024.03.16-2024.04.14)/multi.xlsx",
             "D:/研究生/毕业论文/数据/主播数据/jieguo_100-500(2024.03.16-2024.04.14)/multi.xlsx",
              "D:/研究生/毕业论文/数据/主播数据/jieguo_10-100(2024.03.16-2024.04.14)/multi.xlsx"     ]


In [6]:
# 读取multi.xlsx文件
multi_df = pd.concat([pd.read_excel(file_path) for file_path in file_multi_paths])

In [7]:
# 根据直播ID合并两个DataFrame
merged_df = pd.merge(df_all, multi_df, on='room_id', how='inner')

数据格式检查

In [8]:
# 创建一个空列表，用于存储格式有问题的 room_id
invalid_room_ids = []

# 遍历 merged_df 中的每一行
for index, row in merged_df.iterrows():
    # 获取当前行的 room_id 和 charts 列数据
    room_id = row['room_id']
    charts_data = row['charts']
    
    # 将单引号替换为双引号
    charts_data = charts_data.replace("'", "\"")
    
    try:
        # 使用 json.loads() 将字符串解析为 JSON 对象
        charts_list = json.loads(charts_data)
    except json.JSONDecodeError:
        # 如果解析失败，则记录该行的 room_id
        invalid_room_ids.append(room_id)
# 打印格式有问题的 room_id
print("格式有问题的 room_id：", len(invalid_room_ids))
print("格式有问题的 room_id：", invalid_room_ids)

In [26]:
# 创建字典用于存储不同等级的 room_id
room_ids_by_level = {}

# 遍历 merged_df 中的每一行
for index, row in merged_df.iterrows():
    # 获取当前行的 room_id 和 level
    room_id = row['room_id']
    level = row['level']
    if room_id in invalid_room_ids:
        # 如果当前 level 不在字典中，则创建一个空列表存储该 level 的 room_id
        if level not in room_ids_by_level:
            room_ids_by_level[level] = []

        # 将当前 room_id 添加到相应的 level 列表中
        room_ids_by_level[level].append(room_id)

# 指定保存路径
save_path = "D:/研究生/毕业论文/数据/有问题直播"

# 遍历 room_ids_by_level 字典，将每个 level 的 room_id 保存到对应的 txt 文件中
for level, room_ids in room_ids_by_level.items():
    # 拼接文件名
    filename = os.path.join(save_path, f"room_ids_level_{level}.txt")
    
    # 将 room_ids 写入到 txt 文件中，每个 room_id 之间用逗号分隔
    with open(filename, 'w') as f:
        f.write(','.join(map(str, room_ids)))

应该是平台的直播数据本身的原因，爬了两次都是同样直播间的问题。

删除错误格式数据

In [9]:
# 删除格式有问题的 room_id 对应的行
merged_df = merged_df[~merged_df['room_id'].isin(invalid_room_ids)]

In [10]:
# 创建一个空列表，用于存储展开后的数据
expanded_data_list = []

# 遍历 merged_df 中的每一行
for index, row in merged_df.iterrows():
    # 获取当前行的 room_id 和 charts 列数据
    room_id = row['room_id']
    level = row['level']
    charts_data = row['charts']
    
    # 将单引号替换为双引号
    charts_data = charts_data.replace("'", "\"")
    
    # 使用 json.loads() 将字符串解析为 JSON 对象
    charts_list = json.loads(charts_data)
    
    # 遍历每个直播间的 charts 数据
    for chart in charts_list:
        # 将每个字典添加到列表中，并添加 room_id
        chart['room_id'] = room_id
        chart['level'] = level
        expanded_data_list.append(chart)


In [11]:
# 将列表转换为 DataFrame
expanded_data = pd.DataFrame(expanded_data_list)

# 将 time_node 列转换为日期时间类型
expanded_data['time_node'] = pd.to_datetime(expanded_data['time_node'], unit='s')

expanded_data['time_node'] = expanded_data['time_node'].dt.tz_localize('UTC').dt.tz_convert('Asia/Shanghai')

# 根据 time_node 列提取日期
expanded_data['date'] = expanded_data['time_node'].dt.date

# 提取小时
expanded_data['hour'] = expanded_data['time_node'].dt.hour

# 提取分钟
expanded_data['minute'] = expanded_data['time_node'].dt.minute

In [12]:
# 列出需要增加的列名
columns_to_increment = ['real_time_user', 'leave_user', 'user_count']

# 将这些列的数据全部加 1
expanded_data[columns_to_increment] = expanded_data[columns_to_increment] + 1


In [24]:

# # 过滤掉日期为2024年3月15日的数据
# expanded_data_filtered = expanded_data[expanded_data['date'] != pd.to_datetime('2024-03-15').date()]

# # 按日期和小时分组并计算每小时的在线主播人数和在线观众人数
# hourly_stats = expanded_data_filtered.groupby(['date', 'hour']).agg({'room_id': 'nunique', 'real_time_user': 'sum'})

# # 重命名列
# hourly_stats.rename(columns={'room_id': 'online_streamers', 'real_time_user': 'online_viewers'}, inplace=True)

# # 打印统计结果
# print(hourly_stats)


## 不同时间粒度的数据处理

In [13]:
# 进行数据筛选，只需要晚上时间段的直播
filtered_data = expanded_data[expanded_data['hour'].isin([18, 19, 20, 21, 22, 23])]

In [25]:
def plot_feature_changes(filtered_data, time_interval='5T', save_path='feature_changes_by_level.png'):
    """
    绘制每个等级的特征变化图，根据指定的时间间隔。
    
    Parameters:
    - filtered_data (pd.DataFrame): 包含数据的 DataFrame。
    - time_interval (str): 时间间隔，格式为 '5T', '10T', '30T' 等。
    - save_path (str): 图像保存路径。
    """
    # 创建数据副本，避免链式赋值问题
    filtered_data_n = filtered_data.copy()
    # 使用 .loc 方法进行赋值，避免链式赋值警告
    filtered_data_n.loc[:, 'time_period'] = filtered_data_n['minute'] // (int(time_interval[:-1])) * int(time_interval[:-1])

    # 计算每个直播间在每个时间段的平均特征，仅包括特定的特征
    average_features = filtered_data_n.groupby(['date', 'hour', 'time_period', 'room_id', 'level'])[['real_time_user', 'leave_user', 'user_count']].mean().reset_index()

    # 将同等级的直播间进行求和
    summed_features = average_features.groupby(['date', 'hour', 'time_period', 'level'])[['real_time_user', 'leave_user', 'user_count']].sum().reset_index()

    # 按月计算每个特征的平均值
    monthly_averages = summed_features.groupby(['hour', 'time_period', 'level']).agg({
        'real_time_user': 'mean',
        'leave_user': 'mean',
        'user_count': 'mean'
    }).reset_index()

    # 将 'time_period' 转换为实际时间格式
    monthly_averages['time_period_str'] = monthly_averages['time_period'].astype(int).astype(str).str.zfill(2) + ':00'
    monthly_averages['time_period'] = pd.to_datetime(monthly_averages['hour'].astype(str) + ':' + monthly_averages['time_period_str'], format='%H:%M:%S')

    # 获取不同的等级
    levels = monthly_averages['level'].unique()
    num_levels = len(levels)

    # 创建一个子图网格，根据等级数量决定行数
    fig, axs = plt.subplots(num_levels, 1, figsize=(20, 4*num_levels), dpi=400, sharex=True)

    # 确保 axs 是一个可迭代的对象
    if num_levels == 1:
        axs = [axs]

    # 为每个等级绘制子图
    for i, level in enumerate(levels):
        level_data = monthly_averages[monthly_averages['level'] == level]
        
        # 绘制特征数据
        axs[i].plot(level_data['time_period'], level_data['real_time_user'], marker='o', label='进场用户', color='blue')
        axs[i].plot(level_data['time_period'], level_data['leave_user'], marker='x', label='离线用户', color='red')
        axs[i].plot(level_data['time_period'], level_data['user_count'], marker='s', label='在线用户', color='green')
        
        # 设置标题和标签
        axs[i].set_title(f'主播等级 {level} - 每{time_interval}特征变化', fontweight='bold', fontproperties=font_prop)
        axs[i].set_ylabel('数量', fontweight='bold', fontproperties=font_prop)
        axs[i].legend()

    # 设置共享的 x 轴标签
    axs[-1].set_xlabel('时间段', fontproperties=font_prop)

    # 设置 x 轴刻度
    axs[-1].xaxis.set_major_locator(mdates.MinuteLocator(interval=int(time_interval[:-1])))
    axs[-1].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

    # 自动旋转 x 轴刻度标签
    plt.setp(axs[-1].get_xticklabels(), rotation=45)

    # 调整布局
    plt.tight_layout()

    # 保存图为文件
    plt.savefig(save_path)

    # 显示图形
    plt.show()


In [26]:
# 示例调用
plot_feature_changes(filtered_data, time_interval='1T', save_path='D:/研究生/毕业论文/草稿/图片/feature_changes_1min_by_level.png')

## 保存数据

In [17]:
time_interval = '10T'
# 使用 .loc 方法进行赋值，避免链式赋值警告
filtered_data_new = filtered_data.copy()
filtered_data_new.loc[:, 'time_period'] = filtered_data_new['minute'] // (int(time_interval[:-1])) * int(time_interval[:-1])

# 计算每个直播间在每个时间段的平均特征，仅包括特定的特征
average_features = filtered_data_new.groupby(['date', 'hour', 'time_period', 'room_id', 'level'])[['real_time_user', 'leave_user', 'user_count']].mean().reset_index()

# 对每个小时内所有直播间的实时用户数取整，然后再求和
average_features['real_time_user'] = average_features['real_time_user'].round().astype(int)
average_features['leave_user'] = average_features['leave_user'].round().astype(int)
average_features['user_count'] = average_features['user_count'].round().astype(int)

# 将同等级的直播间进行求和
summed_features = average_features.groupby(['date', 'hour', 'time_period', 'level'])[['real_time_user', 'leave_user', 'user_count']].sum().reset_index()

# 获取所有唯一的等级
levels = summed_features['level'].unique()

# 对每个等级的数据进行提取
for level in levels:
    # 提取当前等级的数据
    level_data = summed_features[summed_features['level'] == level]

    # 删除不需要的列
    level_data = level_data.drop(columns=['level'])
    
    # 保存到 CSV 文件
    output_file = f'./处理数据/hourly_stats_level_{level}.csv'
    level_data.to_csv(output_file, index=False)
    print(f'Saved data for level {level} to {output_file}')


In [18]:
data_folder = './处理数据'

# 处理每个级别的数据
for level in [1, 2, 3]:  # 假设你有3个级别，调整为实际需要的级别
    # 构建文件名
    file_name = f'hourly_stats_level_{level}.csv'
    file_path = os.path.join(data_folder, file_name)
    
    # 读取数据
    hourly_stats = pd.read_csv(file_path)
    
    # 重命名列
    hourly_stats.columns = ['date', 'hour', 'time_period', 'real_time_user', 'leave_user', 'user_count']
    
    # 确保数据按照日期、小时和时间段排序
    hourly_stats = hourly_stats.sort_values(by=['date', 'hour', 'time_period'])
    
    # 创建匹配后的数据
    merged_data = pd.DataFrame()

    # 定义时间间隔的步长
    time_intervals = list(range(0, 60, 10))  # 每隔10分钟
    for i in range(len(time_intervals) - 1):
        # 当前时间段和下一个时间段的分钟数
        current_time_period = time_intervals[i]
        next_time_period = time_intervals[i + 1]
        
        # 获取当前时间段和下一时间段的数据
        current_time_data = hourly_stats[hourly_stats['time_period'] == current_time_period]
        next_time_data = hourly_stats[hourly_stats['time_period'] == next_time_period]
        
        # 合并数据，确保同一天、同一小时的数据匹配
        merged_time_data_1 = pd.merge(current_time_data, next_time_data, on=['date', 'hour'], suffixes=('', '_t'))

        # 只保留匹配成功的行
        if not merged_time_data_1.empty:
            merged_data = pd.concat([merged_data, merged_time_data_1], axis=0)
    
    # 当前时间段和下一个时间段的分钟数
    current_time_period = time_intervals[5]
    next_time_period = time_intervals[0]

    # 获取当前时间段和下一时间段的数据
    current_time_data = hourly_stats[hourly_stats['time_period'] == current_time_period]
    next_time_data = hourly_stats[hourly_stats['time_period'] == next_time_period].copy()

    # 确保 next_time_data 的小时数比 current_time_data 的小时数多 1
    next_time_data['hour'] -= 1
    # 处理有点问题

    # 合并数据，确保同一天且 next_time_data 的小时比 current_time_data 的小时多 1
    merged_time_data_2 = pd.merge(
        current_time_data,
        next_time_data,
        on=['date', 'hour'],  # 现在两个数据集的小时相同，但实际上 next_time_data 的小时比 current_time_data 多 1
        suffixes=('', '_t')
    )

    # 只保留匹配成功的行
    if not merged_time_data_2.empty:
        merged_data = pd.concat([merged_data, merged_time_data_2], axis=0)
    
    # 丢弃不必要的列（如 'time_period_t' 列）
    merged_data = merged_data.drop(columns=['time_period_t'])
    merged_data = merged_data.drop(columns=['date'])

    # 根据 hour 和 time_period 生成索引
    index_mapping = {
        (18, 0): 0, (18, 10): 1, (18, 20): 2, (18, 30): 3, (18, 40): 4, (18, 50): 5,
        (19, 0): 6, (19, 10): 7, (19, 20): 8, (19, 30): 9, (19, 40): 10, (19, 50): 11,
        (20, 0): 12, (20, 10): 13, (20, 20): 14, (20, 30): 15, (20, 40): 16, (20, 50): 17,
        (21, 0): 18, (21, 10): 19, (21, 20): 20, (21, 30): 21, (21, 40): 22, (21, 50): 23,
        (22, 0): 24, (22, 10): 25, (22, 20): 26, (22, 30): 27, (22, 40): 28, (22, 50): 29,
        (23, 0): 30, (23, 10): 31, (23, 20): 32, (23, 30): 33, (23, 40): 34, (23, 50): 35,
    }

    # 应用索引映射
    merged_data['index'] = merged_data.apply(lambda row: index_mapping.get((row['hour'], row['time_period']), -1), axis=1)

    # 将索引列移动到第一列
    merged_data = merged_data[['index'] + [col for col in merged_data.columns if col != 'index']]


    # 丢弃不必要的列（如 'time_period_t' 列）
    merged_data = merged_data.drop(columns=['hour'])
    merged_data = merged_data.drop(columns=['time_period'])


    # 保存处理后的数据
    output_file_path = os.path.join(data_folder, f'processed_hourly_stats_level_{level}.csv')
    merged_data.to_csv(output_file_path, index=False)
    print(f'Processed data saved for level {level} to {output_file_path}')

In [14]:
# 定义数据文件夹路径
data_folder = './处理数据'

# 假设你有 3 个等级
levels = [1, 2, 3]  
level_change_rate_data = {}

for level in levels:
    # 构建文件路径
    file_path = os.path.join(data_folder, f'hourly_stats_level_{level}.csv')
    
    # 读取当前等级的数据
    level_data = pd.read_csv(file_path)
    
    # 确保数据按照日期、小时和时间段排序
    level_data = level_data.sort_values(by=['date', 'hour', 'time_period'])
    
    # 按照 'hour', 'time_period' 分组，计算 'real_time_user' 特征的平均值
    avg_data = level_data.groupby(['hour', 'time_period'])[['real_time_user']].mean().reset_index()
    
    # 计算变化率
    avg_data['next_real_time_user'] = avg_data['real_time_user'].shift(-1)
    avg_data['change_rate'] = (avg_data['next_real_time_user'] - avg_data['real_time_user']) / avg_data['real_time_user']
    
    # 处理时间跨小时的情况：当前的50分钟与下一小时的0分钟
    for idx, row in avg_data.iterrows():
        if row['time_period'] == 50:  # 如果是50分钟
            # 找到下一行的时间是下一小时的0分钟的数据
            next_index = idx + 1
            if next_index < len(avg_data):
                next_row = avg_data.iloc[next_index]
                if next_row['hour'] == (row['hour'] + 1) % 24 and next_row['time_period'] == 0:
                    avg_data.at[idx, 'change_rate'] = (
                        next_row['real_time_user'] - row['real_time_user']
                    ) / row['real_time_user']
            else:
                # 如果没有合适的下一行，则设置为NaN
                avg_data.at[idx, 'change_rate'] = float('nan')
    
    # 删除不需要的列
    avg_data = avg_data.drop(columns=['next_real_time_user'])

    avg_data = avg_data.drop(columns=['hour'])
    avg_data = avg_data.drop(columns=['time_period'])
    avg_data = avg_data.drop(columns=['real_time_user'])

    # 创建索引列
    avg_data['index'] = avg_data.reset_index().index+1
    
    # 将索引列移动到第一列
    avg_data = avg_data[['index'] + [col for col in avg_data.columns if col != 'index']]

    
    # 保存每个等级的变化率数据
    output_file = os.path.join(data_folder, f'change_rate_real_time_user_level_{level}.csv')
    avg_data.to_csv(output_file, index=False)
    print(f'Saved change rate data for level {level} to {output_file}')

    # 将结果存储到字典中，以便后续使用
    level_change_rate_data[level] = avg_data

## average

In [30]:
# # 将时间戳转换为日期和小时
# expanded_data['hour'] = pd.to_datetime(expanded_data['time_node'], unit='s').dt.hour

# # 过滤掉日期为2024年3月15日的数据
# expanded_data_filtered = expanded_data[expanded_data['date'] != pd.to_datetime('2024-03-15').date()]

# 按日期、小时和room_id分组并计算每小时每个房间的在线主播人数和在线观众人数
hourly_stats = expanded_data.groupby(['date', 'hour', 'room_id']).agg({'real_time_user': 'mean'}).reset_index()
print(hourly_stats)
# 对每个小时内所有直播间的实时用户数取整，然后再求和
hourly_stats['real_time_user'] = hourly_stats['real_time_user'].round().astype(int)
hourly_stats = hourly_stats.groupby(['date', 'hour']).agg({'room_id': 'count', 'real_time_user': 'sum'})
print(hourly_stats)
# 重命名列
hourly_stats.rename(columns={'room_id': 'online_streamers', 'real_time_user': 'online_viewers'}, inplace=True)

# 打印统计结果

print(hourly_stats)


In [39]:
# 创建画布和子图
fig, axs = plt.subplots(2, 1, figsize=(12, 10), dpi=200, sharex=True)

# 绘制每天每小时的在线主播人数的折线图
for day, data in hourly_stats.groupby(level=0):
    data['online_streamers'].plot(ax=axs[0], marker='o', label=f'Day {day}')

# 设置第一个子图的图例、标题和轴标签
# axs[0].legend()
axs[0].set_title('Hourly Online Streamers')
axs[0].set_ylabel('Count')
# axs[0].grid(True)
axs[0].set_xticks(range(24))

# 绘制每天每小时的在线观众人数的折线图
for day, data in hourly_stats.groupby(level=0):
    data['online_viewers'].plot(ax=axs[1], marker='x', label=f'Day {day}')

# 设置第二个子图的图例、标题和轴标签
# axs[1].legend()
axs[1].set_title('Hourly Online Viewers')
axs[1].set_xlabel('Hour')
axs[1].set_ylabel('Count')
# axs[1].grid(True)
axs[1].set_xticks(range(0, 25, 5))
axs[1].set_xticklabels(range(0, 25, 5))

# 调整布局
plt.tight_layout()
plt.show()


可以看出，主播和观众的峰值位置保持一致，发展趋势也保持一致。同时值得一提的是，随着数据量的增多，可以发现每天的主播、观众都在一条曲线上稳定地上下波动。

排除每个小时直播场次的差异，每个小时下每场直播间的在线观众也是有所差异的。不应该取平均值，这里再取平均值的含义，每个点都变为某一天的某个小时下，每场直播的平均观众。而我需要的是每个小时有多少场直播，平台总的在线观众。

In [41]:
# 按小时分组计算平均值
hourly_avg = hourly_stats.groupby('hour').mean().reset_index()

# 对平均值取整
hourly_avg['online_streamers'] = hourly_avg['online_streamers'].round().astype(int)
hourly_avg['online_viewers'] = hourly_avg['online_viewers'].round().astype(int)

# 将结果保存到 CSV 文件中
hourly_avg.to_csv('D:/研究生/毕业论文/数据/处理后数据/hourly_avg_stats.csv', index=False)


## max

In [42]:
# 按日期、小时和room_id分组并计算每小时每个房间的在线主播人数和在线观众人数
hourly_stats = expanded_data_filtered.groupby(['date', 'hour', 'room_id']).agg({'real_time_user': 'max'}).reset_index()

# 对每个小时内所有直播间的实时用户数取整，然后再求和
hourly_stats['real_time_user'] = hourly_stats['real_time_user'].round().astype(int)
hourly_stats = hourly_stats.groupby(['date', 'hour']).agg({'room_id': 'nunique', 'real_time_user': 'sum'})

# 重命名列
hourly_stats.rename(columns={'room_id': 'online_streamers', 'real_time_user': 'online_viewers'}, inplace=True)

# 打印统计结果
print(hourly_stats)

In [43]:
# 创建画布和子图
fig, axs = plt.subplots(2, 1, figsize=(12, 10), dpi=200, sharex=True)

# 绘制每天每小时的在线主播人数的折线图
for day, data in hourly_stats.groupby(level=0):
    data['online_streamers'].plot(ax=axs[0], marker='o', label=f'Day {day}')

# 设置第一个子图的图例、标题和轴标签
# axs[0].legend()
axs[0].set_title('Hourly Online Streamers')
axs[0].set_ylabel('Count')
# axs[0].grid(True)
axs[0].set_xticks(range(24))

# 绘制每天每小时的在线观众人数的折线图
for day, data in hourly_stats.groupby(level=0):
    data['online_viewers'].plot(ax=axs[1], marker='x', label=f'Day {day}')

# 设置第二个子图的图例、标题和轴标签
# axs[1].legend()
axs[1].set_title('Hourly Online Viewers')
axs[1].set_xlabel('Hour')
axs[1].set_ylabel('Count')
# axs[1].grid(True)
axs[1].set_xticks(range(0, 25, 5))
axs[1].set_xticklabels(range(0, 25, 5))

# 调整布局
plt.tight_layout()
plt.show()

再取平均值看直播

In [44]:
# 按小时分组计算平均值
hourly_max = hourly_stats.groupby('hour').mean().reset_index()

# 对平均值取整
hourly_max['online_streamers'] = hourly_max['online_streamers'].round().astype(int)
hourly_max['online_viewers'] = hourly_max['online_viewers'].round().astype(int)

# 将结果保存到 CSV 文件中
hourly_max.to_csv('D:/研究生/毕业论文/数据/处理后数据/hourly_max_stats.csv', index=False)


# 处理后数据（不合格）

## 主播和总观众人数

In [45]:
# 创建画布和子图
fig, axs = plt.subplots(2, 1, figsize=(12, 10), dpi=200, sharex=True)

# 绘制主播数的对比图
axs[0].plot(hourly_avg['hour'], hourly_avg['online_streamers'], label='Average Streamers', marker='o')
axs[0].plot(hourly_max['hour'], hourly_max['online_streamers'], label='Max Streamers', marker='x')
axs[0].set_title('Comparison of Streamers')
axs[0].set_ylabel('Count')
axs[0].legend()

# 绘制观众数的对比图
axs[1].plot(hourly_avg['hour'], hourly_avg['online_viewers'], label='Average Viewers', marker='o')
axs[1].plot(hourly_max['hour'], hourly_max['online_viewers'], label='Max Viewers', marker='x')
axs[1].set_title('Comparison of Viewers')
axs[1].set_xlabel('Hour')
axs[1].set_ylabel('Count')
axs[1].legend()

# 显示图形
plt.tight_layout()
plt.show()


## 提取各类观众分布

In [46]:
# 按日期、小时和room_id分组并计算每小时每个房间的在线主播人数和在线观众人数
hourly_stats = expanded_data_filtered.groupby(['date', 'hour', 'room_id']).agg({'real_time_user': 'mean','leave_user': 'mean','user_count': 'mean'}).reset_index()

# 对每个小时内所有直播间的实时用户数取整，然后再求和
hourly_stats['real_time_user'] = hourly_stats['real_time_user'].round().astype(int)
hourly_stats['leave_user'] = hourly_stats['leave_user'].round().astype(int)
hourly_stats['user_count'] = hourly_stats['user_count'].round().astype(int)
# hourly_stats = hourly_stats.groupby(['hour']).agg({'room_id': 'nunique', 'real_time_user': 'sum'})

# 重命名列
# hourly_stats.rename(columns={'room_id': 'online_streamers', 'real_time_user': 'online_viewers'}, inplace=True)

# 打印统计结果
print(hourly_stats)

In [43]:
# 将数据框保存为 CSV 文件
hourly_stats.to_csv('D:/研究生/毕业论文/数据/处理后数据/avg_data.csv', index=False)

In [44]:
# 按日期、小时和room_id分组并计算每小时每个房间的在线主播人数和在线观众人数
hourly_stats = expanded_data_filtered.groupby(['date', 'hour', 'room_id']).agg({'real_time_user': 'max','leave_user': 'max','user_count': 'max'}).reset_index()

# 对每个小时内所有直播间的实时用户数取整，然后再求和
hourly_stats['real_time_user'] = hourly_stats['real_time_user'].round().astype(int)
hourly_stats['leave_user'] = hourly_stats['leave_user'].round().astype(int)
hourly_stats['user_count'] = hourly_stats['user_count'].round().astype(int)
# hourly_stats = hourly_stats.groupby(['hour']).agg({'room_id': 'nunique', 'real_time_user': 'sum'})

# 重命名列
# hourly_stats.rename(columns={'room_id': 'online_streamers', 'real_time_user': 'online_viewers'}, inplace=True)

# 打印统计结果
print(hourly_stats)

In [45]:
# 将数据框保存为 CSV 文件
hourly_stats.to_csv('D:/研究生/毕业论文/数据/处理后数据/max_data.csv', index=False)

网络的研究有可能没有y，可能看网络结构的变化模式。双层网络这种网络结构或子网络结构的变化是否会对最终的观点的偏好影响。节点和边的特征改变，导致子网络特征的改变。

# 正确处理后的数据

In [47]:
# 读取三个xlsx文件，假设文件名分别为file1.xlsx, file2.xlsx, file3.xlsx
file_paths = ["D:/研究生/毕业论文/数据/主播数据/jieguo_1000(2024.03.16-2024.04.14)/info.xlsx", 
             "D:/研究生/毕业论文/数据/主播数据/jieguo_500-1000(2024.03.16-2024.04.14)/info.xlsx",
             "D:/研究生/毕业论文/数据/主播数据/jieguo_100-500(2024.03.16-2024.04.14)/info.xlsx",
             "D:/研究生/毕业论文/数据/主播数据/jieguo_10-100(2024.03.16-2024.04.14)/info.xlsx"]

In [48]:
# 读取三个xlsx文件，假设文件名分别为file1.xlsx, file2.xlsx, file3.xlsx
file_multi_paths = ["D:/研究生/毕业论文/数据/主播数据/jieguo_1000(2024.03.16-2024.04.14)/multi.xlsx", 
             "D:/研究生/毕业论文/数据/主播数据/jieguo_500-1000(2024.03.16-2024.04.14)/multi.xlsx",
             "D:/研究生/毕业论文/数据/主播数据/jieguo_100-500(2024.03.16-2024.04.14)/multi.xlsx",
             "D:/研究生/毕业论文/数据/主播数据/jieguo_10-100(2024.03.16-2024.04.14)/multi.xlsx"]

In [49]:
# 读取三个xlsx文件，假设文件名分别为file1.xlsx, file2.xlsx, file3.xlsx
file_detail_paths = ["D:/研究生/毕业论文/数据/主播数据/jieguo_1000(2024.03.16-2024.04.14)/detail.xlsx", 
             "D:/研究生/毕业论文/数据/主播数据/jieguo_500-1000(2024.03.16-2024.04.14)/detail.xlsx",
             "D:/研究生/毕业论文/数据/主播数据/jieguo_100-500(2024.03.16-2024.04.14)/detail.xlsx",
              "D:/研究生/毕业论文/数据/主播数据/jieguo_10-100(2024.03.16-2024.04.14)/detail.xlsx" ]

In [50]:
# 读取三个xlsx文件，假设文件名分别为file1.xlsx, file2.xlsx, file3.xlsx
file_charts_paths = ["D:/研究生/毕业论文/数据/主播数据/jieguo_1000(2024.03.16-2024.04.14)/charts.xlsx", 
             "D:/研究生/毕业论文/数据/主播数据/jieguo_500-1000(2024.03.16-2024.04.14)/charts.xlsx",
             "D:/研究生/毕业论文/数据/主播数据/jieguo_100-500(2024.03.16-2024.04.14)/charts.xlsx",
              "D:/研究生/毕业论文/数据/主播数据/jieguo_10-100(2024.03.16-2024.04.14)/charts.xlsx"]

In [51]:
# 读取三个xlsx文件，假设文件名分别为file1.xlsx, file2.xlsx, file3.xlsx
file_analysis_paths = ["D:/研究生/毕业论文/数据/主播数据/jieguo_1000(2024.03.16-2024.04.14)/analysis.xlsx", 
             "D:/研究生/毕业论文/数据/主播数据/jieguo_500-1000(2024.03.16-2024.04.14)/analysis.xlsx",
             "D:/研究生/毕业论文/数据/主播数据/jieguo_100-500(2024.03.16-2024.04.14)/analysis.xlsx",
             "D:/研究生/毕业论文/数据/主播数据/jieguo_10-100(2024.03.16-2024.04.14)/analysis.xlsx"]

In [52]:
# 创建一个空DataFrame用于存储所有数据
df_all = pd.DataFrame()

# 循环读取每个文件并合并数据
# 假设文件名中包含等级信息，从文件路径中提取等级
for file_path in file_paths:
    if '10-100' in file_path:
        level = '3'
    elif '100-500' in file_path:
        level = '2'
    elif '500-1000' in file_path:
        level = '1'
    elif '1000' in file_path:
        level = '1'
    else:
        level = 'Unknown'  # 处理未知等级的文件
    # 读取xlsx文件
    df = pd.read_excel(file_path)
    
    # 添加等级列
    df['level'] = level
    
    # 合并数据
    df_all = pd.concat([df_all, df], ignore_index=True)

# 将 begin_time 和 room_finish_time 列的时间戳转换为 pandas 的日期时间格式
df_all['begin_time'] = pd.to_datetime(df_all['begin_time'], unit='s')
df_all['room_finish_time'] = pd.to_datetime(df_all['room_finish_time'], unit='s')

# 将日期时间列的时区从 UTC 转换为北京时间
df_all['begin_time'] = df_all['begin_time'].dt.tz_localize('UTC').dt.tz_convert('Asia/Shanghai')
df_all['room_finish_time'] = df_all['room_finish_time'].dt.tz_localize('UTC').dt.tz_convert('Asia/Shanghai')

# 提取日期作为新的列
df_all['begin_date'] = df_all['begin_time'].dt.date
df_all['end_date'] = df_all['room_finish_time'].dt.date

# 将时间戳转换为日期和小时
df_all['begin_hour'] = pd.to_datetime(df_all['begin_time'], unit='s').dt.hour
df_all['end_hour'] = pd.to_datetime(df_all['room_finish_time'], unit='s').dt.hour

# 计算直播时长
df_all['duration'] = df_all['room_finish_time'] - df_all['begin_time']

# 如果你想要的是以小时为单位的直播时长，可以将时间差转换为小时
df_all['duration_hours'] = df_all['duration'] / pd.Timedelta(hours=1)

# 选择感兴趣的列
columns_of_interest = ['room_id', 'talent_id', 'follower_count', 'following_count',
                       'average_residence_time', 'total_user', 'begin_time', 'room_finish_time', 'average_user_count',
                       'like_count', 'duration_hours', 'level', 'is_take_product', 'begin_date', 'end_date','increment_follower_count',
                       'begin_hour', 'end_hour'
                      ]

# 将指定列保存
df_info = df_all[columns_of_interest]


In [53]:
# 读取multi.xlsx文件
df_multi = pd.concat([pd.read_excel(file_path) for file_path in file_multi_paths])
df_multi = df_multi[['room_id','charts']]
df_detail = pd.concat([pd.read_excel(file_path) for file_path in file_detail_paths])
df_detail = df_detail[['room_id', 'barrage_count', 'interaction_percent']]
df_charts = pd.concat([pd.read_excel(file_path) for file_path in file_charts_paths])
df_charts = df_charts[['room_id', 'total_count']]
df_analysis = pd.concat([pd.read_excel(file_path) for file_path in file_analysis_paths])
df_analysis = df_analysis[['room_id', 'age', 'female', 'male', 'province']]

In [54]:
# 根据 room_id 合并 DataFrame
merged_df = pd.merge(df_info, df_multi, on='room_id', how='inner')
merged_df = pd.merge(merged_df, df_detail, on='room_id', how='inner')
merged_df = pd.merge(merged_df, df_charts, on='room_id', how='inner')
merged_df = pd.merge(merged_df, df_analysis, on='room_id', how='inner')

## 直播持续时间的分布

In [55]:
# 创建画布和子图
plt.figure(figsize=(8, 6))

# 使用Seaborn绘制直方图
sns.histplot(merged_df['duration_hours'], bins=30, color='skyblue', edgecolor='black')

# 添加标题和轴标签
plt.title('Distribution of Duration')
plt.xlabel('Duration')
plt.ylabel('Frequency')

# 显示图形
plt.show()


In [56]:
merged_df['duration_hours'].describe()

一个主播的上线和下线应该根据直播的开始时间和结束时间来确定，duration_hours只能说明一个在线时长问题。

## 主播的上线和离线分布

删除有问题的直播

In [57]:
file_problem_paths = ["D:/研究生/毕业论文/数据/有问题直播/room_ids_level_Level 1.txt",
                      "D:/研究生/毕业论文/数据/有问题直播/room_ids_level_Level 2.txt",
                      "D:/研究生/毕业论文/数据/有问题直播/room_ids_level_Level 3.txt",
                      "D:/研究生/毕业论文/数据/有问题直播/room_ids_level_Level 4.txt"
                     ]
# 读取所有问题文件中的 room_id
problem_room_ids = []
for file_problem_path in file_problem_paths:
    with open(file_problem_path, 'r') as file:
        # 逐行读取文件中的 room_id，并添加到列表中
        for line in file:
            room_ids = line.strip().split(',')
            problem_room_ids.extend(room_ids)

# 将 room_id 转换为字符串类型
problem_room_ids = list(map(str, problem_room_ids))

# 从 merged_df 中过滤掉包含问题 room_id 的行
merged_df = merged_df[~merged_df['room_id'].isin(problem_room_ids)]

# # 打印过滤后的 DataFrame
# print(merged_df)

In [58]:
# # 过滤掉日期为2024年3月15日的数据
# merged_df = merged_df[merged_df['begin_date'] != pd.to_datetime('2024-03-15').date()]

# 按日期和小时分组并计算每小时的在线主播人数和在线观众人数
hourly_stats = merged_df.groupby(['begin_date', 'begin_hour']).agg({'room_id': 'nunique'})

# 重命名列
hourly_stats.rename(columns={'room_id': 'online_streamers'}, inplace=True)

# 打印统计结果
print(hourly_stats)


In [59]:
# 创建画布和子图
fig, ax = plt.subplots(figsize=(12, 6), dpi=200)

# 遍历每一天的数据
for day, data in hourly_stats.groupby(level=0):
    # 绘制每天每小时的在线主播人数和在线观众人数的折线图
    data['online_streamers'].plot(ax=ax, marker='o', label=f'Day {day} - Online Streamers')

# 设置图例、标题和轴标签
# ax.legend()
ax.set_title('Hourly New Online Streamers')
ax.set_xlabel('Hour')
ax.set_ylabel('New Online Streamers')
# ax.grid(True)

# 设置横坐标轴刻度和标签
ax.set_xticks(range(0, 25, 5))
ax.set_xticklabels(range(0, 25, 5))


# 显示图形
plt.tight_layout()
plt.show()

这个和后面的图有问题，因为每一天的某个小时小有缺失的值，它没有画出缺失的值。平均图是对的

In [60]:
# 按小时分组计算平均值
hourly_avg = hourly_stats.groupby('begin_hour').mean().reset_index()
hourly_avg['online_streamers'] = hourly_avg['online_streamers'].round().astype(int)

In [61]:
# 创建画布和子图
fig, ax = plt.subplots(figsize=(12, 6), dpi=200)

ax.plot(hourly_avg['begin_hour'], hourly_avg['online_streamers'], label='Average Streamers', marker='o')

# 设置图例、标题和轴标签
# ax.legend()
ax.set_title('Hourly Average New Online Streamers')
ax.set_xlabel('Hour')
ax.set_ylabel('New Average Online Streamers')
# ax.grid(True)

# 设置横坐标轴刻度和标签
ax.set_xticks(range(0, 25, 5))
ax.set_xticklabels(range(0, 25, 5))


# 显示图形
plt.tight_layout()
plt.show()

In [62]:
# 按日期和小时分组并计算每小时的在线主播人数和在线观众人数
hourly_stats_off = merged_df.groupby(['end_date', 'end_hour']).agg({'room_id': 'nunique'})

# 重命名列
hourly_stats_off.rename(columns={'room_id': 'offline_streamers'}, inplace=True)

# 打印统计结果
print(hourly_stats_off)


In [63]:
# 按小时分组计算平均值
hourly_avg_off = hourly_stats_off.groupby('end_hour').mean().reset_index()
hourly_avg_off['offline_streamers'] = hourly_avg_off['offline_streamers'].round().astype(int)

In [64]:
# 创建画布和子图
fig, ax = plt.subplots(figsize=(12, 6), dpi=200)

ax.plot(hourly_avg_off['end_hour'], hourly_avg_off['offline_streamers'], label='Average Streamers', marker='o')

# 设置图例、标题和轴标签
# ax.legend()
ax.set_title('Hourly Average New Offline Streamers')
ax.set_xlabel('Hour')
ax.set_ylabel('New Average Offline Streamers')
# ax.grid(True)

# 设置横坐标轴刻度和标签
ax.set_xticks(range(0, 25, 5))
ax.set_xticklabels(range(0, 25, 5))


# 显示图形
plt.tight_layout()
plt.show()

In [65]:
#统计一个小时内一直直播的直播数

import pandas as pd

# 创建一个空的字典来存储每个小时范围内直播数
hourly_live_count = {}

# 遍历数据框中的每一行
for index, row in merged_df.iterrows():
    # 计算直播时长（以小时为单位）
    duration_hours = row['room_finish_time'] - row['begin_time']
    
    # 如果直播时长大于1小时，则统计每个小时内一直在直播的直播数
    if duration_hours > pd.Timedelta(hours=1):
        # 生成直播开始时间和结束时间之间的小时范围
        hour_range = pd.date_range(start=row['begin_time'], end=row['room_finish_time'], freq='H')
        
        # 统计每个小时范围内的直播数
        for hour in hour_range[1:]:
            # 提取日期和小时部分作为键
            hour_key = (hour.date(), hour.hour)
            
            # 更新直播数
            if hour_key not in hourly_live_count:
                hourly_live_count[hour_key] = 1
            else:
                hourly_live_count[hour_key] += 1

# 将字典转换为数据框
hourly_live_count_df = pd.DataFrame(list(hourly_live_count.items()), columns=['begin_time', 'same_streamers'])

# 将起始时间和结束时间拆分为两列，并设置为索引
hourly_live_count_df[['begin_date', 'begin_hour']] = pd.DataFrame(hourly_live_count_df['begin_time'].tolist(), index=hourly_live_count_df.index)
hourly_live_count_df = hourly_live_count_df.set_index(['begin_date', 'begin_hour'])

# 打印结果
print(hourly_live_count_df)


In [66]:
# 删除某列
hourly_live_count_df.drop(columns=['begin_time'], inplace=True)

In [67]:
# 按小时分组计算平均值
hourly_live_count_avg = hourly_live_count_df.groupby('begin_hour').mean().reset_index()
hourly_live_count_avg['same_streamers'] = hourly_live_count_avg['same_streamers'].round().astype(int)

In [68]:
# 创建画布和子图
fig, ax = plt.subplots(figsize=(12, 6), dpi=200)

ax.plot(hourly_live_count_avg['begin_hour'], hourly_live_count_avg['same_streamers'], label='Average Streamers', marker='o')

# 设置图例、标题和轴标签
# ax.legend()
ax.set_title('Hourly Average New Same Streamers')
ax.set_xlabel('Hour')
ax.set_ylabel('New Average Same Streamers')
# ax.grid(True)

# # 设置横坐标轴刻度和标签
# ax.set_xticks(range(0, 25, 5))
# ax.set_xticklabels(range(0, 25, 5))


# 显示图形
plt.tight_layout()
plt.show()

In [69]:
# 筛选出开始时间和结束时间在同一天，并且在同一小时的直播
same_day_same_hour = merged_df[(merged_df['begin_time'].dt.date == merged_df['room_finish_time'].dt.date) & 
                               (merged_df['begin_time'].dt.hour == merged_df['room_finish_time'].dt.hour)]

# 按日期和小时分组并计算每小时的直播数量
hourly_stats_same_hour = same_day_same_hour.groupby([same_day_same_hour['begin_time'].dt.date, 
                                                     same_day_same_hour['room_finish_time'].dt.hour]).agg({'room_id': 'nunique'})

hourly_stats_same_hour.rename(columns={'room_id': 'same_streamers'}, inplace=True)

# 打印统计结果
print(hourly_stats_same_hour)

In [70]:
# 按小时分组计算平均值
hourly_avg_same = hourly_stats_same_hour.groupby('room_finish_time').mean().reset_index()
hourly_avg_same['same_streamers'] = hourly_avg_same['same_streamers'].round().astype(int)

In [71]:
# 创建画布和子图
fig, ax = plt.subplots(figsize=(12, 6), dpi=200)

ax.plot(hourly_avg_same['room_finish_time'], hourly_avg_same['same_streamers'], label='Average Streamers', marker='o')

# 设置图例、标题和轴标签
# ax.legend()
ax.set_title('Hourly Average New Same Streamers')
ax.set_xlabel('Hour')
ax.set_ylabel('New Average Same Streamers')
# ax.grid(True)

# # 设置横坐标轴刻度和标签
# ax.set_xticks(range(0, 25, 5))
# ax.set_xticklabels(range(0, 25, 5))


# 显示图形
plt.tight_layout()
plt.show()

In [72]:
# 计算总在线直播并取整,包括begin_hour在这个小时，end_hour在这个小时，以及两者同时在这个小时，和一直在这个小时
hourly_avg_stats_df = (hourly_avg['online_streamers'] + hourly_avg_off['offline_streamers'] - hourly_avg_same['same_streamers']
                      +hourly_live_count_avg['same_streamers']).round().astype(int)

In [73]:
file_path = "D:/研究生/毕业论文/数据/处理后数据/hourly_avg_stats.csv"
hourly_avg_stats_df_1 = pd.read_csv(file_path)

In [74]:
# 创建画布和子图
fig, ax = plt.subplots(figsize=(12, 6), dpi=200)

# 绘制观众数的对比图
ax.plot(hourly_avg['begin_hour'], hourly_avg['online_streamers'], label='Average Online Streamers', marker='o')
ax.plot(hourly_avg_off['end_hour'], hourly_avg_off['offline_streamers'], label='Average Offline Streamers', marker='x')
ax.plot(hourly_avg_same['room_finish_time'], hourly_avg_same['same_streamers'], label='Average Short Streamers', marker='s')
ax.plot(hourly_live_count_avg['begin_hour'], hourly_live_count_avg['same_streamers'], label='Average Long Streamers', marker='p')
ax.plot(hourly_avg_same['room_finish_time'], hourly_avg_stats_df, label='Average Streamers', marker='+')
ax.plot(hourly_avg_same['room_finish_time'], hourly_avg_stats_df_1['online_streamers'], label='Average Streamers 1', marker='*')
ax.set_title('Comparison of Streamers')
ax.set_xlabel('Hour')
ax.set_ylabel('Count')
ax.legend()

# 显示图形
plt.tight_layout()
plt.show()


可以根据主播上线和离线的结果，计算每个小时新加入主播的个数，以及每个小时退出主播的概率。

In [25]:
# 创建 DataFrame 存储数据
data = {
    'begin_hour': hourly_avg['begin_hour'],
    'online_streamers': hourly_avg['online_streamers'],
    'offline_streamers': hourly_avg_off['offline_streamers'],
    'short_streamers': hourly_avg_same['same_streamers'],
    'long_streamers': hourly_live_count_avg['same_streamers'],
    'average_streamers': hourly_avg_stats_df,
    'average_streamers_1': hourly_avg_stats_df_1['online_streamers']
}

df = pd.DataFrame(data)

# 保存 DataFrame 到 CSV 文件
df.to_csv('D:/研究生/毕业论文/数据/处理后数据/streamers_data.csv', index=False)


## 观众的在线、下线分布

连接到4.2节的数据

In [75]:
# 按日期、小时和room_id分组并计算每小时每个房间的在线主播人数和在线观众人数
hourly_stats = expanded_data_filtered.groupby(['date', 'hour', 'room_id']).agg({'real_time_user': 'mean', 'leave_user': 'mean', 'user_count': 'mean'}).reset_index()
# 对每个小时内所有直播间的实时用户数取整，然后再求和
hourly_stats['real_time_user'] = hourly_stats['real_time_user'].round().astype(int)
hourly_stats['leave_user'] = hourly_stats['leave_user'].round().astype(int)
hourly_stats['user_count'] = hourly_stats['user_count'].round().astype(int)
hourly_stats = hourly_stats.groupby(['date', 'hour']).agg({'real_time_user': 'sum', 'leave_user': 'sum', 'user_count': 'sum'})

# # 重命名列
# hourly_stats.rename(columns={'room_id': 'online_streamers', 'real_time_user': 'online_viewers'}, inplace=True)

# 打印统计结果

print(hourly_stats)

In [76]:
# 创建画布和子图
fig, axs = plt.subplots(3, 1, figsize=(12, 10), dpi=200, sharex=True)

# 绘制每天每小时的在线主播人数的折线图
for day, data in hourly_stats.groupby(level=0):
    data['real_time_user'].plot(ax=axs[0], marker='o', label=f'Day {day}')

# 设置第一个子图的图例、标题和轴标签
# axs[0].legend()
axs[0].set_title('Hourly Real Time User')
axs[0].set_ylabel('Count')
# axs[0].grid(True)
axs[0].set_xticks(range(24))

# 绘制每天每小时的在线观众人数的折线图
for day, data in hourly_stats.groupby(level=0):
    data['leave_user'].plot(ax=axs[1], marker='x', label=f'Day {day}')

# 设置第二个子图的图例、标题和轴标签
# axs[1].legend()
axs[1].set_title('Hourly Leave User')
axs[1].set_ylabel('Count')
# axs[1].grid(True)
axs[1].set_xticks(range(24))

# 绘制每天每小时的在线观众人数的折线图
for day, data in hourly_stats.groupby(level=0):
    data['user_count'].plot(ax=axs[2], marker='*', label=f'Day {day}')

# 设置第二个子图的图例、标题和轴标签
# axs[1].legend()
axs[2].set_title('Hourly User Count')
axs[2].set_xlabel('Hour')
axs[2].set_ylabel('Count')
# axs[1].grid(True)
axs[2].set_xticks(range(0, 25, 5))
axs[2].set_xticklabels(range(0, 25, 5))

# 调整布局
plt.tight_layout()
plt.show()


In [77]:
# 按小时分组计算平均值
hourly_avg = hourly_stats.groupby('hour').mean().reset_index()

# 对平均值取整
hourly_avg['real_time_user'] = hourly_avg['real_time_user'].round().astype(int)
hourly_avg['leave_user'] = hourly_avg['leave_user'].round().astype(int)
hourly_avg['user_count'] = hourly_avg['user_count'].round().astype(int)

In [78]:
# 创建画布和子图
fig, ax = plt.subplots(figsize=(12, 6), dpi=200)

# 绘制观众数的对比图
ax.plot(hourly_avg['hour'], hourly_avg['real_time_user'], label='Average Real Time User', marker='o')
ax.plot(hourly_avg['hour'], hourly_avg['leave_user'], label='Average Leave User', marker='x')
ax.plot(hourly_avg['hour'], hourly_avg['user_count'], label='Average User Count', marker='s')
ax.set_title('Comparison of Users')
ax.set_xlabel('Hour')
ax.set_ylabel('Count')
ax.legend()

# 显示图形
plt.tight_layout()
plt.show()

In [16]:
# 创建 DataFrame 存储数据
data = {
    'begin_hour': hourly_avg['hour'],
    'real_time_user': hourly_avg['real_time_user'],
    'leave_user': hourly_avg['leave_user'],
    'user_count': hourly_avg['user_count']
}

df = pd.DataFrame(data)

# 保存 DataFrame 到 CSV 文件
df.to_csv('D:/研究生/毕业论文/数据/处理后数据/user_data.csv', index=False)


详细数据

In [12]:

# 按日期、小时和room_id分组并计算每小时每个房间的在线主播人数和在线观众人数
hourly_stats = expanded_data_filtered.groupby(['date', 'hour', 'room_id', 'level']).agg({'real_time_user': 'mean', 'leave_user': 'mean',
                                                                                'user_count': 'mean'}).reset_index()
# 对每个小时内所有直播间的实时用户数取整，然后再求和
hourly_stats['real_time_user'] = hourly_stats['real_time_user'].round().astype(int)
hourly_stats['leave_user'] = hourly_stats['leave_user'].round().astype(int)
hourly_stats['user_count'] = hourly_stats['user_count'].round().astype(int)
hourly_stats = hourly_stats.groupby(['date', 'hour', 'level']).agg({'real_time_user': 'sum', 'leave_user': 'sum', 'user_count': 'sum'})

In [13]:
hourly_stats

In [14]:
# 按小时分组计算平均值
hourly_avg = hourly_stats.groupby(['hour', 'level']).mean().reset_index()

# 对平均值取整
hourly_avg['real_time_user'] = hourly_avg['real_time_user'].round().astype(int)
hourly_avg['leave_user'] = hourly_avg['leave_user'].round().astype(int)
hourly_avg['user_count'] = hourly_avg['user_count'].round().astype(int)

In [15]:
hourly_avg

In [16]:
# 计算每个小时所有等级的real_time_user总和
hourly_totals = hourly_avg.groupby('hour')['real_time_user'].sum().reset_index()
hourly_totals.rename(columns={'real_time_user': 'total_real_time_user'}, inplace=True)

# 合并hourly_avg和hourly_totals以获取每小时的总real_time_user
hourly_avg = pd.merge(hourly_avg, hourly_totals, on='hour')

# 计算每个等级的real_time_user占该小时总和的比例
hourly_avg['level_proportion'] = hourly_avg['real_time_user'] / hourly_avg['total_real_time_user']

# 透视表，将每个小时的各等级比例转换为列
hourly_proportions = hourly_avg.pivot_table(index='hour', columns='level', values='level_proportion', fill_value=0).reset_index()

# 重命名列以便于理解
hourly_proportions.columns.name = None  # 移除列名
hourly_proportions.columns = ['hour', 'level1_proportion', 'level2_proportion', 'level3_proportion']

# 显示结果
print(hourly_proportions)


In [17]:
# 保存 DataFrame 到 CSV 文件
hourly_proportions.to_csv('D:/研究生/毕业论文/数据/处理后数据/user_data_xi.csv', index=False)

# 主播带货品牌之间的关系

## 不等等级之间的主播

In [3]:
# 创建一个空DataFrame用于存储所有数据
df_all = pd.DataFrame()

# 循环读取每个文件并合并数据
# 假设文件名中包含等级信息，从文件路径中提取等级
for file_path in file_paths:
    if '10-100' in file_path:
        level = '3'
    elif '100-500' in file_path:
        level = '2'
    elif '500-1000' in file_path:
        level = '1'
    elif '1000' in file_path:
        level = '1'
    else:
        level = 'Unknown'  # 处理未知等级的文件
    # 读取xlsx文件
    df = pd.read_excel(file_path)
    
    # 添加等级列
    df['level'] = level
    
    # 合并数据
    df_all = pd.concat([df_all, df], ignore_index=True)

# 将 begin_time 和 room_finish_time 列的时间戳转换为 pandas 的日期时间格式
df_all['begin_time'] = pd.to_datetime(df_all['begin_time'], unit='s')
# df_all['room_finish_time'] = pd.to_datetime(df_all['room_finish_time'], unit='s')

# 将日期时间列的时区从 UTC 转换为北京时间
df_all['begin_time'] = df_all['begin_time'].dt.tz_localize('UTC').dt.tz_convert('Asia/Shanghai')
# df_all['room_finish_time'] = df_all['room_finish_time'].dt.tz_localize('UTC').dt.tz_convert('Asia/Shanghai')

# 提取日期作为新的列
df_all['begin_date'] = df_all['begin_time'].dt.date
# df_all['end_date'] = df_all['room_finish_time'].dt.date

# 将时间戳转换为日期和小时
df_all['begin_hour'] = pd.to_datetime(df_all['begin_time'], unit='s').dt.hour
# df_all['end_hour'] = pd.to_datetime(df_all['room_finish_time'], unit='s').dt.hour

# # 计算直播时长
# df_all['duration'] = df_all['room_finish_time'] - df_all['begin_time']

# # 如果你想要的是以小时为单位的直播时长，可以将时间差转换为小时
# df_all['duration_hours'] = df_all['duration'] / pd.Timedelta(hours=1)

# 选择感兴趣的列
columns_of_interest = ['room_id', 'talent_id', 'begin_time', 'level', 'is_take_product', 'begin_date', 'begin_hour']

# 将指定列保存
df_info = df_all[columns_of_interest]


In [4]:
# 统计每个level下的直播间数量
level_counts = df_info['level'].value_counts()

# 计算总的直播间数量
total_rooms = level_counts.sum()

# 计算每个level的直播间数量占总直播间数量的比例
level_percentages = level_counts / total_rooms

# 打印结果
print(level_percentages)


In [4]:
# 读取三个xlsx文件，假设文件名分别为file1.xlsx, file2.xlsx, file3.xlsx
file_filters_paths = ["D:/研究生/毕业论文/数据/主播数据/jieguo_1000(2024.03.16-2024.04.14)/filters.xlsx", 
             "D:/研究生/毕业论文/数据/主播数据/jieguo_500-1000(2024.03.16-2024.04.14)/filters.xlsx",
             "D:/研究生/毕业论文/数据/主播数据/jieguo_100-500(2024.03.16-2024.04.14)/filters.xlsx",
             "D:/研究生/毕业论文/数据/主播数据/jieguo_10-100(2024.03.16-2024.04.14)/filters.xlsx"]

In [5]:
df_filters = pd.concat([pd.read_excel(file_path) for file_path in file_filters_paths])
df_filters = df_filters[['room_id','brand']]

In [6]:
# 根据直播ID合并两个DataFrame
merged_df = pd.merge(df_info, df_filters, on='room_id', how='inner')

In [7]:
invalid_room_ids = []

# 遍历每个直播间的品牌信息
for _, row in merged_df.iterrows():
    brands = row['brand']
    room_id = row['room_id']
    # 将字符串中的 None 替换为 null
    brands = brands.replace('None', 'null')
    
    # 将单引号替换为双引号
    brands = brands.replace("\'", "\"")
    
    #name:“Dr.Ci:Labo/城野医生”针对字符串的冒号需要去除，json认为这是字典的一对键值
    # 使用正则表达式去除 `\` 之前，前一个双引号 `"` 和斜杠之间的内容
    brands = re.sub(r'"[^"]+/[^"]+"', 'null', brands)
    
    #针对name:"L"null，
    # 使用正则表达式匹配符合条件的内容并进行替换
    brands = re.sub(r':\s*"[^"]*null', ':null', brands)
    
    #针对name:"I"MONE"
    brands = re.sub(r'"[^",:]*"[^",:]*"', 'null', brands)
    
    #针对name:nullnull
    brands = re.sub(r'nullnull', 'null', brands)

    #针对}'
    # 判断字符串的最后一位是否是]
    if brands[-1] == '}' and brands[-1] != ']':
        brands += ']'
        
    #针对z最后没有}]'  
    if brands[-1] != '}' and brands[-1] != ']':
        brands += '}]'
        
    #其余情况为数据爬取不全造成的原因，给予删除
    
    try:
        # 使用 json.loads() 将字符串解析为 JSON 对象
        brands = json.loads(brands)
    except json.JSONDecodeError:
        # 如果解析失败，则记录该行的 room_id
        invalid_room_ids.append(room_id)
# 打印格式有问题的 room_id
print("格式有问题的 room_id：", len(invalid_room_ids))
# print("格式有问题的 room_id：", invalid_room_ids)

In [8]:
# 删除格式有问题的 room_id 对应的行
merged_df = merged_df[~merged_df['room_id'].isin(invalid_room_ids)]

In [9]:
# 创建一个字典，用于存储统计结果
statistics = defaultdict(lambda: defaultdict(lambda: defaultdict(set)))

# 遍历每个直播间的品牌信息
for _, row in merged_df.iterrows():
    begin_date = row['begin_date']
    begin_hour = row['begin_hour']
    level = row['level']
    brands = row['brand']
    
    # 将字符串中的 None 替换为 null
    brands = brands.replace('None', 'null')
    
    # 将单引号替换为双引号
    brands = brands.replace("\'", "\"")
    
    #name:“Dr.Ci:Labo/城野医生”针对字符串的冒号需要去除，json认为这是字典的一对键值
    # 使用正则表达式去除 `\` 之前，前一个双引号 `"` 和斜杠之间的内容
    brands = re.sub(r'"[^"]+/[^"]+"', 'null', brands)
    
    #针对name:"L"null，
    # 使用正则表达式匹配符合条件的内容并进行替换
    brands = re.sub(r':\s*"[^"]*null', ':null', brands)
    
    #针对name:"I"MONE"
    brands = re.sub(r'"[^",:]*"[^",:]*"', 'null', brands)
    
    #针对name:nullnull
    brands = re.sub(r'nullnull', 'null', brands)

    #针对}'
    # 判断字符串的最后一位是否是]
    if brands[-1] == '}' and brands[-1] != ']':
        brands += ']'
        
    #针对z最后没有}]'  
    if brands[-1] != '}' and brands[-1] != ']':
        brands += '}]'
        
    # 先转换为 JSON 格式
    brands = json.loads(brands)
    
    # 提取每个品牌信息中的商品集合并进行统计
    for brand_info in brands:
        name = brand_info.get('id')
        if name is not None:
            statistics[begin_date][begin_hour][level].add(name)


In [10]:
statistics

In [11]:
# 创建字典来存储每个时间段内不同等级之间商品的交集和并集
hourly_intersection = defaultdict(lambda: defaultdict(set))
hourly_union = defaultdict(lambda: defaultdict(set))

# 计算每个时间段内不同等级之间商品的交集和并集
for date, hour_data in statistics.items():
    for hour, level_data in hour_data.items():
        # 将每个等级的商品集合转换为集合列表
        level_sets = [set(products) for products in level_data.values()]
        
        # 计算不同等级之间商品的交集和并集
        for i in range(len(level_sets)):
            for j in range(i+1, len(level_sets)):
                level1 = list(level_data.keys())[i]
                level2 = list(level_data.keys())[j]
                intersection = level_sets[i].intersection(level_sets[j])
                union = level_sets[i].union(level_sets[j])
                
                # 存储交集和并集
                hourly_intersection[date][(hour, level1, level2)] = intersection
                hourly_union[date][(hour, level1, level2)] = union

# 创建字典来存储每个时间段内不同等级之间商品交集和并集的数量
hourly_intersection_count = defaultdict(lambda: defaultdict(int))
hourly_union_count = defaultdict(lambda: defaultdict(int))

# 计算每个时间段内不同等级之间商品交集和并集的数量
for date, hour_data in hourly_intersection.items():
    for hour_level, intersection in hour_data.items():
        union = hourly_union[date][hour_level]
        intersection_count = len(intersection)
        union_count = len(union)
        
        # 存储交集和并集的数量
        hourly_intersection_count[date][hour_level] = intersection_count
        hourly_union_count[date][hour_level] = union_count

# 计算每一天每小时下商品交集除以并集的概率
hourly_probability = defaultdict(lambda: defaultdict(float))
for date, hour_data in hourly_intersection_count.items():
    for hour_level, intersection_count in hour_data.items():
        union_count = hourly_union_count[date][hour_level]
        
        # 计算概率，避免除数为0的情况
        if union_count != 0:
            probability = intersection_count / union_count
        else:
            probability = 0
        
        # 存储概率
        hourly_probability[date][hour_level] = probability


In [12]:
hourly_probability

In [19]:
# 将嵌套字典展开
flat_hourly_probability = []
for date, hour_data in hourly_probability.items():
    for hour_level, probability in hour_data.items():
        hour, level1, level2 = hour_level
        flat_hourly_probability.append({'begin_date': date, 'begin_hour': hour, 'level1': level1, 'level2': level2, 'probability': probability})

# 创建 DataFrame
hourly_probability_df = pd.DataFrame(flat_hourly_probability)

# 打印结果
print(hourly_probability_df)


In [18]:
date

In [20]:
# 按日期和小时分组并计算每小时的在线主播人数和在线观众人数
# hourly_probability_df = hourly_probability_df.groupby(['begin_hour', 'level1', 'level2']).mean().reset_index()
hourly_probability_df = hourly_probability_df.groupby(['begin_hour', 'level1', 'level2']).agg({'probability': 'mean'})

In [21]:
# 生成所有可能的组合
possible_combinations = pd.DataFrame(list(itertools.product(range(24), ['1', '2'], [ '2', '3'])))

# 去除level1=level2的组合
possible_combinations = possible_combinations[possible_combinations[1] != possible_combinations[2]]

# 将其与当前 DataFrame 合并
hourly_probability_filled = pd.merge(possible_combinations, hourly_probability_df, left_on=[0, 1, 2], right_on=['begin_hour', 'level1', 'level2'], how='left')

# 填充缺失值为0
hourly_probability_filled['probability'].fillna(0, inplace=True)

# 重新命名列名
hourly_probability_filled.rename(columns={0: 'hour'}, inplace=True)
# 重新命名列名
hourly_probability_filled.rename(columns={1: 'l1'}, inplace=True)
# 重新命名列名
hourly_probability_filled.rename(columns={2: 'l2'}, inplace=True)

# 打印结果
print(hourly_probability_filled)


In [24]:
# 假设 hourly_probability_filled 已经存在并且包含所有小时和组合的概率数据
# 检查每个小时下，是否包含所有三种组合
hours = range(24)
combinations = [('1', '2'), ('1', '3'), ('2', '3')]

# 补全缺失的数据
for hour in hours:
    for l1, l2 in combinations:
        if not ((hourly_probability_filled['hour'] == hour) & 
                (hourly_probability_filled['l1'] == l1) & 
                (hourly_probability_filled['l2'] == l2)).any():
            hourly_probability_filled = hourly_probability_filled.append(
                {'begin_hour': hour, 'l1': l1, 'l2': l2, 'probability': 0.0},
                ignore_index=True
            )

# 提取数据并创建新的 DataFrame
level1_2 = hourly_probability_filled[(hourly_probability_filled['l1'] == '1') & (hourly_probability_filled['l2'] == '2')][['hour', 'probability']].rename(columns={'probability': 'probability_1_2'})
level1_3 = hourly_probability_filled[(hourly_probability_filled['l1'] == '1') & (hourly_probability_filled['l2'] == '3')][['hour', 'probability']].rename(columns={'probability': 'probability_1_3'})
level2_3 = hourly_probability_filled[(hourly_probability_filled['l1'] == '2') & (hourly_probability_filled['l2'] == '3')][['hour', 'probability']].rename(columns={'probability': 'probability_2_3'})

# 合并到一个 DataFrame 中
merged_df = pd.merge(level1_2, level1_3, on='hour', how='outer')
merged_df = pd.merge(merged_df, level2_3, on='hour', how='outer')

# 填充缺失值
merged_df = merged_df.fillna(0)

# 打印结果
print(merged_df)

# 将合并后的 DataFrame 保存为 CSV 文件
csv_file_path = 'D:/研究生/毕业论文/数据/处理后数据/streamers_guanxi_between.csv'
merged_df.to_csv(csv_file_path, index=False)


In [99]:
# 提取数据
hours = range(0,24)
level1_2 = hourly_probability_filled[(hourly_probability_filled['l1'] == '1') & (hourly_probability_filled['l2'] == '2')]['probability']
level1_3 = hourly_probability_filled[(hourly_probability_filled['l1'] == '1') & (hourly_probability_filled['l2'] == '3')]['probability']
level2_3 = hourly_probability_filled[(hourly_probability_filled['l1'] == '2') & (hourly_probability_filled['l2'] == '3')]['probability']


# 创建画布和子图
fig, ax = plt.subplots(figsize=(12, 6), dpi=200)

# 绘制观众数的对比图
# 绘制图形
ax.plot(hours, level1_2, label='level1_2', marker='o')
ax.plot(hours, level1_3, label='level1_3', marker='x')
ax.plot(hours, level2_3, label='level2_3', marker='s')
ax.set_title('Hourly Probability by Level1 and Level2')
ax.set_xlabel('Hour')
ax.set_ylabel('Probability')
ax.legend()

# 显示图形
plt.tight_layout()
plt.show()


In [95]:
len(level2_3)

## 同等级之间的主播

In [16]:
def calculate_average_probability(statistics):
    result = defaultdict(lambda: defaultdict(float))
    
    for begin_hour in range(24):  # 假设 begin_hour 的范围是 0 到 23
        for level in ['1', '2', '3']:  # 假设 level 的可能值
            product_sets = []
            for begin_date in statistics:
                if begin_hour in statistics[begin_date] and level in statistics[begin_date][begin_hour]:
                    product_sets.append(statistics[begin_date][begin_hour][level])
            
#             if product_sets:
            probabilities = []
            for set1, set2 in itertools.combinations(product_sets, 2):
                intersection = set1.intersection(set2)
                union = set1.union(set2)
                if union:
                    probability = len(intersection) / len(union)
                else:
                    probability = 0
                probabilities.append(probability)

            if probabilities:
                average_probability = sum(probabilities) / len(probabilities)
            else:
                average_probability = 0

            result[begin_hour][level] = average_probability

    return result


In [17]:
result = calculate_average_probability(statistics)

In [18]:
result

In [19]:
import csv

def save_to_csv(result, filename):
    with open(filename, 'w', newline='') as csvfile:
        fieldnames = ['hour', 'level1', 'level2', 'level3']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for hour in result:
            row = {'hour': hour}
            for level in result[hour]:
                row[f'level{level}'] = result[hour][level]
            writer.writerow(row)

# 调用保存函数
save_to_csv(result, 'D:/研究生/毕业论文/数据/处理后数据/streamers_guanxi_on.csv')
